In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

#load train model, scaler pickle, 
#load pickel file to do prediction
model=load_model('churn_model.h5')
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

#example input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
print(geo_encoded_df)

gender_encoded = label_encoder_gender.transform([input_data['Gender']])
gender_encoded_df = pd.DataFrame(gender_encoded, columns=['Gender'])
print(gender_encoded_df)
input_data = pd.DataFrame([input_data])  # convert to dataframe

# Remove 'Geography' column and add encoded columns
input_data = input_data.drop("Geography", axis=1)
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data = pd.concat([input_data.reset_index(drop=True), geo_encoded_df], axis=1)
# Reorder columns to match scaler's expected order
expected_columns = scaler.feature_names_in_
print("Expected columns:", expected_columns)

input_data = input_data[expected_columns]
input_scaled = scaler.transform(input_data)


predictions=model.predict(input_scaled)
print("Prediction:", predictions)

if predictions[0][0]>0.5:
    print("Customer will leave the bank")
else:
    print("Customer will stay with the bank")



   Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0
   Gender
0       1
Expected columns: ['CreditScore' 'Gender' 'Age' 'Tenure' 'Balance' 'NumOfProducts'
 'HasCrCard' 'IsActiveMember' 'EstimatedSalary' 'Geography_France'
 'Geography_Germany' 'Geography_Spain']
1/1 [==============================] - 0s 22ms/step
Prediction: [[0.03510314]]
Customer will stay with the bank


/Users/khushalidesai/Desktop/ANN Classification/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
